# Data collection
This file contains the code to collect a list of bird species from GBIF in the Ede area and download related bird song audio from Xeno Canto.

GBIF.org (01 March 2023) GBIF Occurrence Download  https://doi.org/10.15468/dl.hzws49

In [1]:
from urllib.request import urlopen, urlretrieve, HTTPError, URLError
import os
import cgi
import duckdb
import pandas as pd
import logging

logging.basicConfig(filename='app.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Connect to Duck DB database file
con = duckdb.connect('data/db/collection.duckdb')

## Functions

In [3]:
def fetch_bird_recordings(search_key, page_number, quality = 'A'):
    print(f'Fetching page {page_number} of {search_key}')
    url = f'https://xeno-canto.org/api/2/recordings?query=q:{quality}+{search_key}&page={page_number}'
    df = pd.read_json(url)
    current_page = df['page'].max()
    number_of_pages = df['numPages'].max()
    df = pd.DataFrame(df.recordings.values.tolist())
    
    con.sql(f'''
        INSERT INTO bird_recordings
        SELECT rec.*, '{search_key}' AS search_key
        FROM df AS rec
        WHERE rec.also = '[]'
    ''')
    
    if current_page < number_of_pages:
        fetch_bird_recordings(search_key, page_number + 1, quality)

In [4]:
def download_audio_data(dest_folder, file_url):
    try:
        remotefile = urlopen(file_url)
        contentdisposition = remotefile.info()['Content-Disposition']
        _, params = cgi.parse_header(contentdisposition)
        filename = f'data/audio/{dest_folder}/{params["filename"]}'

        os.makedirs(os.path.dirname(filename), exist_ok=True)
        urlretrieve(file_url, filename)

        print(f'Downloaded file: {filename}')
    except HTTPError as err:
        log_msg = f'Failed to download file at {file_url}, because of error code: {err.code}'
        print(log_msg)
        logging.error(log_msg)
    
    except URLError as err:
        log_msg = f'Failed to download file at {file_url}, because of error reason: {err.reason}'
        print(log_msg)
        logging.error(log_msg)
    

## Pipeline

In [48]:
# Load observerd species based on GBIF data from Ede to a grouped pandas dataframe
df = pd.read_csv('data/csv/ede_bird_occurrence_pp.csv', sep='\t')
species = df[['species', 'individualCount']] \
            .fillna(1) \
            .groupby('species') \
            .agg(['sum','count']) \
            .pipe(lambda x: x.set_axis(x.columns.map('_'.join), axis=1)) \
            .reset_index()

In [58]:
# Store the dataframe data into the database
con.sql('''
    CREATE TABLE bird_observations_ede AS
    SELECT
        species,
        REPLACE(LOWER(species), ' ', '+') AS search_key,
        individualCount_sum               AS observation_sum,
        individualCount_count             AS observation_cnt
    FROM
        species
    ORDER BY
        individualCount_count DESC
''')

In [122]:
# Take the top 10 species and fetch their search keys to loop over
search_keys = con.sql('''
    SELECT search_key
    FROM bird_observations_ede
    LIMIT 50
''').fetchnumpy()

con.sql('''
    SELECT *
    FROM bird_observations_ede
    LIMIT 50
''').to_df()

,species,search_key,observation_sum,observation_cnt
0,Parus major,parus+major,1351.0,729
1,Fringilla coelebs,fringilla+coelebs,3835.0,575
2,Dryocopus martius,dryocopus+martius,493.0,452
3,Buteo buteo,buteo+buteo,548.0,434
4,Sitta europaea,sitta+europaea,559.0,427
5,Dendrocopos major,dendrocopos+major,483.0,383
6,Lanius excubitor,lanius+excubitor,344.0,336
7,Emberiza citrinella,emberiza+citrinella,670.0,331
8,Turdus merula,turdus+merula,503.0,321
9,Alauda arvensis,alauda+arvensis,784.0,282


In [5]:
# Take the species that don't exist in the Google Model but do in the Edese bos area
search_keys = con.sql('''
    SELECT search_key
    FROM bird_observations_ede
    WHERE search_key IN ('galerida+cristata', 'picus+viridis', 'phasianus+colchicus')
''').fetchnumpy()

con.sql('''
    SELECT *
    FROM bird_observations_ede
    WHERE search_key IN ('galerida+cristata', 'picus+viridis', 'phasianus+colchicus')
''').to_df()

,species,search_key,observation_sum,observation_cnt
0,Galerida cristata,galerida+cristata,4.0,1
1,Picus viridis,picus+viridis,17.0,16
2,Phasianus colchicus,phasianus+colchicus,12.0,12


In [6]:
# Loop over search keys
for search_key in search_keys['search_key']:
    fetch_bird_recordings(search_key, 1, 'A')

Fetching page 1 of galerida+cristata
Fetching page 1 of picus+viridis
Fetching page 1 of phasianus+colchicus


In [11]:
con.sql('''
    SELECT search_key, COUNT(*) AS count
    FROM bird_recordings
    WHERE search_key IN ('galerida+cristata', 'picus+viridis', 'phasianus+colchicus')
    GROUP BY search_key
    ORDER BY count DESC
''').to_df()

,search_key,count
0,picus+viridis,256
1,galerida+cristata,199
2,phasianus+colchicus,139


In [12]:
df = con.sql('''
    SELECT REPLACE(search_key, '+', '_') AS dest_folder, file
    FROM bird_recordings
    WHERE search_key IN ('galerida+cristata', 'picus+viridis', 'phasianus+colchicus')
''').to_df()

In [13]:
df

,dest_folder,file
0,galerida_cristata,https://xeno-canto.org/803678/download
1,galerida_cristata,https://xeno-canto.org/798070/download
2,galerida_cristata,https://xeno-canto.org/793022/download
3,galerida_cristata,https://xeno-canto.org/792605/download
4,galerida_cristata,https://xeno-canto.org/792604/download
...,...,...
589,phasianus_colchicus,https://xeno-canto.org/65778/download
590,phasianus_colchicus,https://xeno-canto.org/41223/download
591,phasianus_colchicus,https://xeno-canto.org/37452/download
592,phasianus_colchicus,https://xeno-canto.org/22692/download


In [14]:
for index, row in df.iterrows():
    download_audio_data(row['dest_folder'], row['file'])

Downloaded file: data/audio/galerida_cristata/XC803678 - Crested Lark - Galerida cristata.mp3
Downloaded file: data/audio/galerida_cristata/XC798070 - Crested Lark - Galerida cristata.wav
Downloaded file: data/audio/galerida_cristata/XC793022 - Crested Lark - Galerida cristata.wav
Downloaded file: data/audio/galerida_cristata/XC792605 - Crested Lark - Galerida cristata.mp3
Downloaded file: data/audio/galerida_cristata/XC792604 - Crested Lark - Galerida cristata.mp3
Downloaded file: data/audio/galerida_cristata/XC792542 - Crested Lark - Galerida cristata.mp3
Downloaded file: data/audio/galerida_cristata/XC792389 - Crested Lark - Galerida cristata.wav
Downloaded file: data/audio/galerida_cristata/XC792388 - Crested Lark - Galerida cristata.wav
Downloaded file: data/audio/galerida_cristata/XC782232 - Crested Lark - Galerida cristata.mp3
Downloaded file: data/audio/galerida_cristata/XC782230 - Crested Lark - Galerida cristata.mp3
Downloaded file: data/audio/galerida_cristata/XC782229 - Cre

In [116]:
# con.sql('''
#     SELECT id
#     FROM bird_recordings
# ''').to_csv('data/csv/ids.csv')

## Load eBird Taxonomy data and join it to our birds of interrest

In [8]:
# Load eBird Taxonomy
ebird = pd.read_excel('data/ebird_taxonomy_v2022.xlsx')
ebird[['SPECIES_CODE', 'SCI_NAME']]

,SPECIES_CODE,SCI_NAME
0,ostric2,Struthio camelus
1,ostric3,Struthio molybdophanes
2,y00934,Struthio camelus/molybdophanes
3,grerhe1,Rhea americana
4,lesrhe2,Rhea pennata
...,...,...
16855,slcgro1,Saltator grossus
16856,bltgro2,Saltator fuliginosus
16857,saltat1,Saltator sp.
16858,passer1,Passeriformes sp.


In [15]:
# Get species that do not map to be able to create a manual mapping file
con.sql('''
    SELECT
        bo.*,
        eb.SPECIES_CODE AS ebird_code
    FROM
        bird_observations_ede AS bo
    LEFT JOIN
        ebird AS eb
    ON
        bo.search_key = LOWER(REPLACE(eb.SCI_NAME, ' ', '+'))
    WHERE
        eb.SPECIES_CODE IS NULL
    ORDER BY
        observation_cnt DESC
''').to_df()

0    Phylloscopus sibillatrix
1            Coloeus monedula
2             Sylvia communis
3               Anas clypeata
4            Anas querquedula
5               Aquila clanga
6              Sylvia curruca
Name: species, dtype: object

In [18]:
# Load Manual mapping
mmap = pd.read_excel('data/manual_mapping_ebird_gbif.xlsx')

In [19]:
mmap

,ebird_code,ebird_name,gbif_name
0,woowar,Phylloscopus sibilatrix,Phylloscopus sibillatrix
1,eurjac,Corvus monedula,Coloeus monedula
2,grewhi1,Curruca communis,Sylvia communis
3,norsho,Spatula clypeata,Anas clypeata
4,gargan,Spatula querquedula,Anas querquedula
5,grseag1,Clanga clanga,Aquila clanga
6,sylvia1,Sylvia/Curruca sp.,Sylvia curruca


In [20]:
con.sql('DROP TABLE bird_observations_ede_sc')

In [21]:
con.sql('''
    CREATE TABLE bird_observations_ede_sc AS
    SELECT
        bo.*,
        COALESCE(eb.SPECIES_CODE, eb2.SPECIES_CODE) AS ebird_code_def
    FROM
        bird_observations_ede AS bo
    LEFT JOIN
        mmap as mp
    ON
        bo.species = mp.gbif_name
    LEFT JOIN
        ebird AS eb
    ON
        bo.search_key = LOWER(REPLACE(eb.SCI_NAME, ' ', '+'))
    LEFT JOIN
        ebird AS eb2
    ON
        mp.ebird_code = eb2.SPECIES_CODE
    WHERE
        ebird_code_def IS NOT NULL
    ORDER BY
        observation_cnt DESC
''')

In [24]:
con.sql('''
    SELECT *
    FROM bird_observations_ede_sc
    WHERE ebird_code_def = 'gargan'
''').to_df()

,species,search_key,observation_sum,observation_cnt,ebird_code_def
0,Anas querquedula,anas+querquedula,1.0,1,gargan


In [6]:
rs = pd.read_csv('data/result.csv')
rs

,frame,time,class,probability
0,0,0:00:00,martit2,0.366938
1,1,0:00:05,gretit1,0.266846
2,2,0:00:10,blutit,0.432775
3,3,0:00:15,shttre1,0.111287
4,4,0:00:20,blutit,0.634578
...,...,...,...,...
715,715,0:59:35,coatit2,0.660683
716,716,0:59:40,coatit2,0.194544
717,717,0:59:45,gretit1,0.277770
718,718,0:59:50,fieldf,0.215344


In [26]:
con.sql('''
    SELECT
        rs.class,
        bo.SCI_NAME,
        COUNT(*) AS aantal_obs
    FROM
        rs
    LEFT JOIN
        ebird AS bo
    ON
        rs.class = bo.SPECIES_CODE
    WHERE
        rs.probability > 0.7
    GROUP BY
        rs.class,
        bo.SCI_NAME
    ORDER BY
        aantal_obs DESC
''').to_df()

,class,SCI_NAME,aantal_obs
0,gretit1,Parus major,44
1,martit2,Poecile palustris,40
2,eurnut2,Sitta europaea,35
3,shttre1,Certhia brachydactyla,22
4,grswoo,Dendrocopos major,20
5,eurrob1,Erithacus rubecula,8
6,tawowl1,Strix aluco,4
7,miswoo1,Dendrocoptes medius,4
8,eurbla,Turdus merula,4
9,blutit,Cyanistes caeruleus,3


In [11]:
con.sql('''
    CREATE TABLE uncommon_obs_ede AS
    SELECT
        rs.*,
        CAST(rs.time AS TIME) - INTERVAL 5 SECOND AS time_m5,
        CAST(rs.time AS TIME) + INTERVAL 5 SECOND AS time_p5,
        bo.SCI_NAME,
    FROM
        rs
    LEFT JOIN
        ebird AS bo
    ON
        rs.class = bo.SPECIES_CODE
    WHERE
        rs.probability > 0.5 AND
        rs.class NOT IN (
            SELECT ebird_code_def
            FROM bird_observations_ede_sc
        )
    ORDER BY
        rs.frame;
''')

In [37]:
con.sql('''
    COPY
        uncommon_obs_ede
    TO
        'data/csv/species_detected_not_in_gbif.csv' (HEADER, DELIMITER ';');
''')

In [34]:
con.sql('''
    SELECT
        species,
        ebird_code_def,
        SUM(observation_sum) AS observation_sum,
        SUM(observation_cnt) AS observation_cnt
    FROM
        bird_observations_ede_sc
    GROUP BY
        species,
        ebird_code_def
    ORDER BY
        observation_cnt DESC;
''').to_df()

,species,ebird_code_def,observation_sum,observation_cnt
0,Parus major,gretit1,1351.0,729.0
1,Fringilla coelebs,comcha,3835.0,575.0
2,Dryocopus martius,blawoo1,493.0,452.0
3,Buteo buteo,combuz1,548.0,434.0
4,Sitta europaea,eurnut2,559.0,427.0
...,...,...,...,...
140,Luscinia megarhynchos,comnig1,1.0,1.0
141,Ficedula albicollis,colfly1,1.0,1.0
142,Saxicola rubetra,whinch1,1.0,1.0
143,Plectrophenax nivalis,snobun,1.0,1.0


In [36]:
con.sql('''
    COPY (
        SELECT
            species,
            ebird_code_def,
            SUM(observation_sum) AS observation_sum,
            SUM(observation_cnt) AS observation_cnt
        FROM
            bird_observations_ede_sc
        GROUP BY
            species,
            ebird_code_def
        ORDER BY
            observation_cnt DESC
    )
    TO
        'data/csv/gbif_species_ede.csv' (HEADER, DELIMITER ';');
''')

## Load meteo data

In [3]:
rain = pd.read_excel(r'data/rain_per_hour_fixed.xlsx')

In [5]:
con.sql('''
    DROP TABLE rain_per_hour
''')

In [6]:
con.sql('''
    CREATE TABLE rain_per_hour AS
    SELECT
        *
    FROM
        rain;
''')

## Experimental code snipets
These snipets were used to experiment and kept for future use.

In [89]:
search_key = 'parus+major'
page_number = 1
url = f'https://xeno-canto.org/api/2/recordings?query=q:A+{search_key}&page={page_number}'

# urlretrieve(url, f'data/json/{search_key}_{page_number}.json')

('data/json/parus+major_1.json', <http.client.HTTPMessage at 0x172e16ac0>)

In [94]:
# Collect recordings
df = pd.read_json(url)
current_page = df['page'].max()
number_of_pages = df['numPages'].max()
df = pd.DataFrame(df.recordings.values.tolist())

In [152]:
# Download audio file
dest_folder = 'test'
file_url = 'https://xeno-canto.org/783426/download'

remotefile = urlopen(file_url)
contentdisposition = remotefile.info()['Content-Disposition']
_, params = cgi.parse_header(contentdisposition)
filename = f'data/audio/{dest_folder}/{params["filename"]}'

os.makedirs(os.path.dirname(filename), exist_ok=True)
urlretrieve(file_url, filename)

('data/audio/test/XC783426 - Australian Masked Owl - Tyto novaehollandiae castanops.wav',
 <http.client.HTTPMessage at 0x1777feca0>)

In [165]:
log_msg = f'Test message'
print(log_msg)
logging.error(log_msg)

Test message


In [123]:
# con.sql(f'''
#     CREATE TABLE bird_recordings AS
#     SELECT rec.*, '{search_key}' AS search_key
#     FROM df_rec AS rec
#     WHERE also = '[]'
# ''')

# con.sql('''
#     DELETE
#     FROM bird_recordings
# ''')

In [8]:
con.sql('''
    DELETE
    FROM bird_recordings
    WHERE search_key IN ('galerida+cristata', 'picus+viridis', 'phasianus+colchicus');
''')

In [7]:
con.close()